In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from hydra import initialize, compose
from rdkit import Chem
from ergochemics.mapping import rc_to_nest, get_reaction_center
from ergochemics.draw import draw_reaction, draw_molecule
from minedatabase.pickaxe import Pickaxe

with initialize(version_base=None, config_path="../configs/filepaths"):
    cfg = compose(config_name="filepaths")

In [4]:
expansion_name = "2_steps_ccm_aa_aplusb_True"
cpd_df = pd.read_parquet(Path(cfg.processed_data) / f"{expansion_name}_compound_metrics.parquet")
cpd_df = cpd_df.groupby(["expansion", "gen"]).agg({"fan_out": ["mean", "std"]})
cpd_df.head(24)

fan_out  \
                                                              mean   
expansion                                          gen               
2_steps_ccm_aa_rules_imt_rules_w_coreactants_ap... 0    164.793103   
                                                   1     60.500000   
2_steps_ccm_aa_rules_mechinferred_dt_15_rules_w... 0    166.689655   
                                                   1     36.062603   
2_steps_ccm_aa_rules_mechinferred_dt_2_rules_w_... 0     86.724138   
                                                   1     21.681499   
2_steps_ccm_aa_rules_mechinferred_dt_3_rules_w_... 0    119.310345   
                                                   1     26.777070   
2_steps_ccm_aa_rules_mechinferred_dt_6_rules_w_... 0    137.034483   
                                                   1     28.583650   
2_steps_ccm_aa_rules_mechinferred_dt_98_rules_w... 0    186.724138   
                                                   1     44.057851   
2_steps_ccm_aa_rules_mechinformed_rules_w_corea... 0    111.896552   
                                                   1     26.923077   
2_steps_ccm_aa_rules_rc_plus_0_rules_w_coreacta... 0    387.931034   
                                                   1    211.192738   
2_steps_ccm_aa_rules_rc_plus_1_rules_w_coreacta... 0     68.068966   
                                                   1     20.415094   
2_steps_ccm_aa_rules_rc_plus_2_rules_w_coreacta... 0     30.034483   
                                                   1     11.021053   
2_steps_ccm_aa_rules_rc_plus_3_rules_w_coreacta... 0     13.758621   
                                                   1      7.352941   
2_steps_ccm_aa_rules_rc_plus_4_rules_w_coreacta... 0     13.137931   
                                                   1      6.560386   

                                                                     
                                                                std  
expansion                                          gen               
2_steps_ccm_aa_rules_imt_rules_w_coreactants_ap... 0     292.132110  
                                                   1      40.682036  
2_steps_ccm_aa_rules_mechinferred_dt_15_rules_w... 0     204.523855  
                                                   1      27.235182  
2_steps_ccm_aa_rules_mechinferred_dt_2_rules_w_... 0     117.871570  
                                                   1      15.475309  
2_steps_ccm_aa_rules_mechinferred_dt_3_rules_w_... 0     154.962231  
                                                   1      20.730161  
2_steps_ccm_aa_rules_mechinferred_dt_6_rules_w_... 0     191.619877  
                                                   1      21.224775  
2_steps_ccm_aa_rules_mechinferred_dt_98_rules_w... 0     261.401429  
                                                   1      37.749986  
2_steps_ccm_aa_rules_mechinformed_rules_w_corea... 0     194.829659  
                                                   1      20.073932  
2_steps_ccm_aa_rules_rc_plus_0_rules_w_coreacta... 0    1015.279094  
                                                   1     183.280402  
2_steps_ccm_aa_rules_rc_plus_1_rules_w_coreacta... 0      75.831548  
                                                   1      13.402387  
2_steps_ccm_aa_rules_rc_plus_2_rules_w_coreacta... 0      35.764590  
                                                   1       5.461734  
2_steps_ccm_aa_rules_rc_plus_3_rules_w_coreacta... 0       6.711691  
                                                   1       3.603326  
2_steps_ccm_aa_rules_rc_plus_4_rules_w_coreacta... 0       7.740082  
                                                   1       3.122884

In [5]:
rxn_df = pd.read_parquet(Path(cfg.processed_data) / f"{expansion_name}_reaction_metrics.parquet")
rxn_df = rxn_df.groupby("expansion").agg({"dxgb_label": "mean"})
rxn_df.head(12)

,dxgb_label
expansion,
2_steps_ccm_aa_rules_imt_rules_w_coreactants_aplusb_True.pk,0.284289
2_steps_ccm_aa_rules_mechinferred_dt_15_rules_w_coreactants_aplusb_True.pk,0.452857
2_steps_ccm_aa_rules_mechinferred_dt_2_rules_w_coreactants_aplusb_True.pk,0.544323
2_steps_ccm_aa_rules_mechinferred_dt_3_rules_w_coreactants_aplusb_True.pk,0.515559
2_steps_ccm_aa_rules_mechinferred_dt_6_rules_w_coreactants_aplusb_True.pk,0.489283
2_steps_ccm_aa_rules_mechinferred_dt_98_rules_w_coreactants_aplusb_True.pk,0.442432
2_steps_ccm_aa_rules_mechinformed_rules_w_coreactants_aplusb_True.pk,0.402511
2_steps_ccm_aa_rules_rc_plus_0_rules_w_coreactants_aplusb_True.pk,0.226116
2_steps_ccm_aa_rules_rc_plus_1_rules_w_coreactants_aplusb_True.pk,0.502010
